In [ ]:
from google.colab import files
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import random
random.seed(420)


url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_test.data'
df_test = pd.read_csv(url, header=None, sep =' ')
df_test = df_test.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.data'
X_train = pd.read_csv(url, header=None, sep =' ')
X_train = X_train.dropna(how='all', axis=1)

url = 'https://raw.githubusercontent.com/kozaka93/2023Z-AutoML/main/homeworks/homework2/artificial_train.labels'
y_train = pd.read_csv(url, header=None)

In [ ]:
from sklearn.preprocessing import LabelEncoder

y_train2 = LabelEncoder().fit_transform(y_train)
y_train2

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([0, 0, 0, ..., 0, 1, 1])

In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest
fs = SelectKBest(score_func=f_classif,k=25)
fit = fs.fit(X_train,y_train2)

In [ ]:
features_score = pd.DataFrame(fit.scores_)
features = pd.DataFrame(X_train.columns)
feature_score = pd.concat([features,features_score],axis=1)
feature_score.columns = ["Input_Features","F_Score"]
print(feature_score.nlargest(25,columns="F_Score"))

     Input_Features     F_Score
475             475  101.556789
241             241   99.524269
336             336   47.045066
64               64   46.194897
128             128   32.627036
105             105   32.368782
338             338   30.081219
48               48   28.444776
378             378   27.233677
442             442   26.474649
472             472   25.413854
453             453   18.405687
493             493   18.217882
323             323    8.392836
424             424    6.638351
282             282    6.462605
411             411    6.220247
205             205    6.007540
296             296    5.975923
496             496    5.697433
55               55    5.647685
204             204    5.616683
430             430    5.476813
119             119    5.431805
136             136    5.280475


In [ ]:
from sklearn.feature_selection import f_classif, SelectKBest
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# anova = SelectKBest(score_func=f_classif,k=15)
# clf = XGBClassifier(random_state=21)

pipe = Pipeline([('anova', SelectKBest(score_func=f_classif,k=15)), ('xgb', XGBClassifier(random_state=21))])
pipe.get_params
# clf.fit(X_train2, y_train2)
# clf.score(X_train2, y_train2)

<bound method Pipeline.get_params of Pipeline(steps=[('anova', SelectKBest(k=15)),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=None,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=None, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_const

In [ ]:
from scipy.stats import uniform, randint


grid_params_rs = [{
    "xgb__n_estimators": randint(1,5000),
    "xgb__learning_rate": uniform(0,10),
    "xgb__max_depth": randint(1,15),
    "xgb__min_child_weight": uniform(0,7)
}]

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

results_rs = pd.DataFrame()
grid = RandomizedSearchCV(pipe,grid_params_rs, scoring='balanced_accuracy', random_state=37, n_iter=50)

In [ ]:
grid.fit(X_train, y_train2)

RandomizedSearchCV(estimator=Pipeline(steps=[('anova', SelectKBest(k=15)),
                                             ('xgb',
                                              XGBClassifier(base_score=None,
                                                            booster=None,
                                                            callbacks=None,
                                                            colsample_bylevel=None,
                                                            colsample_bynode=None,
                                                            colsample_bytree=None,
                                                            device=None,
                                                            early_stopping_rounds=None,
                                                            enable_categorical=False,
                                                            eval_metric=None,
                                                            feature_types=None,
                                                            gamma=None,
                                                            grow_policy=None,
                                                            importance_type=N...
                   param_distributions=[{'xgb__learning_rate': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7da2de7bd5d0>,
                                         'xgb__max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da2de7bee60>,
                                         'xgb__min_child_weight': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7da2de7be590>,
                                         'xgb__n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7da2de700c70>}],
                   random_state=37, scoring='balanced_accuracy')

In [ ]:
results_rs["params"] = grid.cv_results_["params"]
results_rs["a_scores"] = grid.cv_results_["mean_test_score"]

In [ ]:
grid.best_estimator_.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('anova', SelectKBest(k=15)),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None,
                               learning_rate=0.38721346767512266, max_bin=None,
                               max_cat_threshold=None, max_cat_to_onehot=None,
                               max_delta_step=None, max_depth=5,
                               max_leaves=None,
                               min_child_weight=0.6010107840065457, m

In [ ]:
grid.best_score_

0.8364999999999998

In [ ]:
grid.best_estimator_.predict_proba(df_test)[:,0]

In [ ]:
results_rs.to_csv("/drive/My Drive/AutoML/hw2/xgb/xgb_pipe_results_rs.csv",index=False)